## Data Cleaning

In [ ]:
import pandas as pd

In [ ]:
usda = pd.read_csv('USDA.csv')
intake = pd.read_csv('intake.csv', header = [0,1])
recipes = pd.read_csv('world_recipes.csv')

usda.size

112928

In [ ]:
len(usda.columns.to_list())

16

In [ ]:
def clean_unit(unit):
  match = re.search(r'\)', unit)
  if match:
    return unit[:match.end()]
  else:
    return unit

new_columns = []
for n, u in intake.columns:
  clean = clean_unit(u)
  new_columns.append((n.strip(), clean.strip()))

intake.columns = pd.MultiIndex.from_tuples(new_columns)
intake.head()

,Life Stage,Vitamin A,Vitamin C,Vitamin D,Vitamin E,Vitamin K,Thiamin,Riboflavin,Niacin,Vitamin B6,...,Potassium,Sodium,Chloride,Total Water,Carbohydrate,Total Fiber,Fat,Linoleic Acid,α-Linolenic Acid,Protein
,units,(μg/d),(mg/d),(μg/d),(mg/d),(μg/d),(mg/d),(mg/d),(mg/d),(mg/d),...,(mg/d),(mg/d),(g/d),(L/d),(g/d),(g/d),(g/d),(g/d),(g/d),(g/d)
0,Infants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0–6 mo,400*,40*,10*,4*,2.0*,0.2*,0.3*,2*,0.1*,...,400*,110*,0.18*,0.7*,60*,ND,31*,4.4*,0.5*,9.1*
2,6–12 mo,500*,50*,10*,5*,2.5*,0.3*,0.4*,4*,0.3*,...,860*,370*,0.57*,0.8*,95*,ND,30*,4.6*,0.5*,11
3,Children,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1–3 y,300,15,15,6,30*,0.5,0.5,6,0.5,...,"2,000*",800*,1.5*,1.3*,130,19*,NDc,7*,0.7*,13


In [ ]:
# remove all asterisks
intake = intake.replace(r'\*', '', regex = True)
intake.columns


MultiIndex([(           'Group',       ''),
            (      'Life Stage',  'units'),
            (       'Vitamin A', '(μg/d)'),
            (       'Vitamin C', '(mg/d)'),
            (       'Vitamin D', '(μg/d)'),
            (       'Vitamin E', '(mg/d)'),
            (       'Vitamin K', '(μg/d)'),
            (         'Thiamin', '(mg/d)'),
            (      'Riboflavin', '(mg/d)'),
            (          'Niacin', '(mg/d)'),
            (      'Vitamin B6', '(mg/d)'),
            (          'Folate', '(μg/d)'),
            (     'Vitamin B12', '(μg/d)'),
            ('Pantothenic Acid', '(mg/d)'),
            (          'Biotin', '(μg/d)'),
            (         'Choline', '(mg/d)'),
            (         'Calcium', '(mg/d)'),
            (        'Chromium', '(μg/d)'),
            (          'Copper', '(μg/d)'),
            (        'Fluoride', '(mg/d)'),
            (          'Iodine', '(μg/d)'),
            (            'Iron', '(mg/d)'),
            (       'Magnesium',

In [ ]:
life_stage_col = ('Life Stage', 'units')
groups = ['Infants', 'Children', 'Males', 'Females', 'Pregnancy', 'Lactation']
intake['Group'] = intake[life_stage_col].where(intake[life_stage_col].isin(groups))
intake['Group'] = intake['Group'].ffill()
intake = intake[~intake[life_stage_col].isin(groups)]
# intake = intake.reset_index()
# intake = intake.drop(columns = 'index')
intake.head()

def move_col(intake, col, pos = 0):
  cols = list(intake.columns)
  cols.insert(pos, cols.pop(cols.index(col)))
  return intake[cols]

intake = move_col(intake, ('Group', ''), 0)
intake.head()


,Group,Life Stage,Vitamin A,Vitamin C,Vitamin D,Vitamin E,Vitamin K,Thiamin,Riboflavin,Niacin,...,Potassium,Sodium,Chloride,Total Water,Carbohydrate,Total Fiber,Fat,Linoleic Acid,α-Linolenic Acid,Protein
,,units,(μg/d),(mg/d),(μg/d),(mg/d),(μg/d),(mg/d),(mg/d),(mg/d),...,(mg/d),(mg/d),(g/d),(L/d),(g/d),(g/d),(g/d),(g/d),(g/d),(g/d)
1,Infants,0–6 mo,400,40,10,4,2.0,0.2,0.3,2,...,400,110,0.18,0.7,60,ND,31,4.4,0.5,9.1
2,Infants,6–12 mo,500,50,10,5,2.5,0.3,0.4,4,...,860,370,0.57,0.8,95,ND,30,4.6,0.5,11
4,Children,1–3 y,300,15,15,6,30,0.5,0.5,6,...,"2,000",800,1.5,1.3,130,19,NDc,7,0.7,13
5,Children,4–8 y,400,25,15,7,55,0.6,0.6,8,...,"2,300","1,000",1.9,1.7,130,25,ND,10,0.9,19
7,Males,9–13 y,600,45,15,11,60,0.9,0.9,12,...,"2,500","1,200",2.3,2.4,130,31,ND,12,1.2,34


In [ ]:
intake.to_csv('intake.csv', index = False)

STOP HERE

In [ ]:
def get_ingredients(meal_description):
  prompt = f"""You are a nutrition planner. Based on the following meal description, list the likely ingredients and rough quantities in JSON format.
  Meal: f"""

In [ ]:
pip install fuzzywuzzy[speedup] openai --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.5 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/d

In [ ]:
## NOT NEEDED

# load 3 main csvs
food = pd.read_csv('food.csv')
nutrient = pd.read_csv('nutrient.csv')
food_nutrient = pd.read_csv('food_nutrient.csv')

# merge food_nutrient with nutrient to get nutrient names
fn = food_nutrient.merge(nutrient, left_on='nutrient_id', right_on='id')

# merge fn again with food to get food names
fn = fn.merge(food[['fdc_id', 'description']], on='fdc_id')

# Pivot to get nutrients as columns
pivoted = fn.pivot_table(
    index='description',
    columns='name',
    values='amount',
    aggfunc='first'
).reset_index()

# Optional: Rename columns like 'Energy' → 'Calories (kcal)'
pivoted.rename(columns={'Energy': 'Calories (kcal)'}, inplace=True)

# Save for later
pivoted.to_csv('cleaned_nutrition_lookup.csv', index=False)
